# Validation - distribution of GT and optimized AP potentials/currents

This notebook evaluates qualitatively the distributions of membrane potentials and transmembrane currents of an action potential over the neuron morphology.

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
import shutil

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import matplotlib.pyplot as plt
from scipy.spatial import distance
import MEAutility as mu
import json
import time
import numpy as np
from pathlib import Path

import pandas as pd
import seaborn as sns
from scipy.spatial.distance import cosine

import multimodalfitting as mf

%matplotlib notebook

In [ ]:
base_path = Path("../..")

In [ ]:
probe_type = "planar"
model_name = "hay_ais" # "hay", "hay_ais", "hay_ais_hillock"
cell_models_folder = base_path / "cell_models"

model_folder = cell_models_folder / f"{model_name}"

In [ ]:
result_folder = base_path / "results" / '211124' 
pkl_file_name = "runs.pkl"

data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

In [ ]:
opt_soma = df_model.query("feature_set == 'soma'")
opt_extra = df_model.query("feature_set == 'extra'")
print(f"Somatic optimizations: {len(opt_soma)}")
print(f"Extra optimizations: {len(opt_extra)}")

In [ ]:
protocols_file = model_folder / "fitting" / "efeatures" / "protocols_BPO_all.json"
features_file = model_folder / "fitting" / "efeatures" / "features_BPO_all.json"

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_name, release=False)
cell_release = mf.create_ground_truth_model(model_name=model_name, release=True)

probe = mf.define_electrode(probe_type=probe_type)

param_names = [param.name for param in cell.params.values() if not param.frozen]

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

In [ ]:
param_names

In [ ]:
params_release

In [ ]:
protocol_for_eap = "IDrest_300"

### Define more recording points 

In [ ]:
extra_kwargs = mf.utils.get_extra_kwargs()

In [ ]:
eva_extra = mf.create_evaluator(
    model_name=model_name,
    feature_set="extra",
    extra_strategy="all",
    protocols_with_lfp=protocol_for_eap,
    **extra_kwargs
)

In [ ]:
mf.plot_cell(eva_extra.cell_model, eva_extra.sim, param_values=params_release)

In [ ]:
positions = np.array([[-62, 828], [-3, 954], [-27, 546], [-27, 85], [134, -28], [-117, -189]])
position_names = ["apical_distal_left", "apical_distal_right", "apical_middle", "apical_proximal", 
                  "basal_right", "basal_left"]

In [ ]:
extra_recordings = mf.utils.extra_recordings_from_positions(cell_release, eva_extra.sim, positions, position_names)

In [ ]:
extra_recordings

In [ ]:
eva_extra = mf.create_evaluator(
    model_name=model_name,
    feature_set="extra",
    extra_strategy="all",
    protocols_with_lfp=protocol_for_eap,
    extra_recordings=dict(IDrest_300=extra_recordings), 
    **extra_kwargs
)

In [ ]:
eva_extra.fitness_protocols["IDrest_300"].recordings

In [ ]:
len(extra_recordings)

# Load protocols and original features

In [ ]:
ais_recording = mf.utils.get_ais_extra_recordings()

In [ ]:
ais_recording

In [ ]:
# eva_extra = mf.create_evaluator(
#     model_name=model_name,
#     feature_set="extra",
#     extra_strategy="all",
#     protocols_with_lfp=protocol_for_eap,
#     extra_recordings=dict(IDrest_300=ais_recording), 
#     **extra_kwargs
# )

In [ ]:
# only run IDrest 300

In [ ]:
idrest = eva_extra.fitness_protocols["IDrest_300"]

In [ ]:
best_soma = np.argmin(opt_soma.best_fitness)
best_sample_soma = opt_soma.iloc[best_soma]
best_params_soma_dict = {k: v for k, v in zip(param_names, best_sample_soma.best_params)}
display("SOMA", best_sample_soma.best_fitness)

opt_all = opt_extra.query("extra_strategy == 'all'")
opt_sections = opt_extra.query("extra_strategy == 'sections'")
opt_single = opt_extra.query("extra_strategy == 'single'")

best_all = np.argmin(opt_all.best_fitness)
best_sections = np.argmin(opt_sections.best_fitness)
best_single = np.argmin(opt_single.best_fitness)

best_sample_all = opt_all.iloc[best_all]
best_params_all_dict = {k: v for k, v in zip(param_names, best_sample_all.best_params)}
display("ALL", best_sample_all.best_fitness)

best_sample_sections = opt_sections.iloc[best_sections]
best_params_sections_dict = {k: v for k, v in zip(param_names, best_sample_sections.best_params)}
display("SECTIONS", best_sample_sections.best_fitness)

best_sample_single = opt_single.iloc[best_single]
best_params_single_dict = {k: v for k, v in zip(param_names, best_sample_single.best_params)}
display("SINGLE", best_sample_single.best_fitness)

In [ ]:
print("Computing RELEASE")
response_release = eva_extra.run_protocol(idrest, params_release)
print("Computing SOMA")
response_soma = eva_extra.run_protocol(idrest, best_params_soma_dict)
print("Computing ALL")
response_all = eva_extra.run_protocol(idrest, best_params_all_dict)
print("Computing SECTIONS")
response_sections = eva_extra.run_protocol(idrest, best_params_sections_dict)
print("Computing SINGLE")
response_single = eva_extra.run_protocol(idrest, best_params_single_dict)

In [ ]:
ms_after = 50

In [ ]:
resp_cut_release = mf.utils.get_peak_cutout(response_release, ms_after=ms_after)
resp_cut_soma = mf.utils.get_peak_cutout(response_soma, ms_after=ms_after)
resp_cut_all = mf.utils.get_peak_cutout(response_all, ms_after=ms_after)
resp_cut_sections = mf.utils.get_peak_cutout(response_sections, ms_after=ms_after)
resp_cut_single = mf.utils.get_peak_cutout(response_single, ms_after=ms_after)

In [ ]:
responses_to_plot = ['IDrest_300.apical_proximal.v', 
                     'IDrest_300.apical_middle.v', 
                     'IDrest_300.apical_distal_left.v',
                     'IDrest_300.apical_distal_right.v',
                     'IDrest_300.basal_left.v',
                     'IDrest_300.basal_right.v']

In [ ]:
distance_arr = []
strategy_arr = []
position_arr = []
for resp_name in responses_to_plot:
    fig, ax = plt.subplots()
    ax.plot(resp_cut_release[resp_name]["time"], resp_cut_release[resp_name]["voltage"], color="k", label="GT")
    ax.plot(resp_cut_soma[resp_name]["time"], resp_cut_soma[resp_name]["voltage"], color="C0", label="SOMA")
    ax.plot(resp_cut_all[resp_name]["time"], resp_cut_all[resp_name]["voltage"], color="C1", label="ALL")
    ax.plot(resp_cut_sections[resp_name]["time"], resp_cut_sections[resp_name]["voltage"], color="C2", 
            label="SECTIONS")
    position = resp_name.split(".")[1]
    dist_soma = cosine(resp_cut_release[resp_name]["voltage"],
                       resp_cut_soma[resp_name]["voltage"])
    dist_all = cosine(resp_cut_release[resp_name]["voltage"],
                      resp_cut_all[resp_name]["voltage"])
    dist_sec = cosine(resp_cut_release[resp_name]["voltage"],
                      resp_cut_sections[resp_name]["voltage"])
    position_arr.append(position)
    distance_arr.append(dist_soma)
    strategy_arr.append("soma")
    position_arr.append(position)
    distance_arr.append(dist_all)
    strategy_arr.append("all")
    position_arr.append(position)
    distance_arr.append(dist_sec)
    strategy_arr.append("sections")
    ax.axvline(1., color="gray", ls="--")
#     ax.plot(resp_cut_single[resp_name]["time"], resp_cut_single[resp_name]["voltage"], color="C3", label="SINGLE")
    ax.legend()
    ax.set_title(resp_name)

df = pd.DataFrame({"strategy": strategy_arr, "distance": distance_arr, "position": position_arr})

In [ ]:
plt.figure()
sns.barplot(data=df, x="strategy", y="distance", hue="position")